# Visualize Active Learning Run 

In [12]:
%load_ext autoreload
%autoreload 2
%cd /mnt/activeCell-ACDC
!ls

import json
import random as rd
import matplotlib.image as mpimg
import cv2
import wandb
import torch
import os
import logging
import pandas as pd
import collections
import matplotlib.pyplot as plt

from detectron2.utils.visualizer import Visualizer
from detectron2.utils.logger import setup_logger
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer

from src.globals import *
from utils.visualization.show_image import show_image
from utils.register_datasets import register_datasets, get_dataset_name
from utils.config_builder import get_config
from utils.notebook_utils import *


from src.test import do_test
from src.active_learning.al_trainer import *
from src.active_learning.mc_dropout_sampler import *


logger = setup_logger(output="./log/main.log",name="null_logger") 
logger.addHandler(logging.NullHandler())
logging.getLogger('detectron2').setLevel(logging.WARNING)
logging.getLogger('detectron2').addHandler(logging.NullHandler())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/mnt/activeCell-ACDC
README.md    data		nohup.out  pipeline_configs  src
ac_acdc_env  jupyter_notebooks	notebooks  requirements.txt  utils
al_output    log		output	   shell_scripts     wandb


In [14]:
dataset = ACDC_LARGE
config_name = "classes_acdc_large_al"
OUTPUT_PATH = "al_output"
strategy = "hybrid"
base_path = os.path.join(OUTPUT_PATH,config_name,strategy)



'al_output/classes_acdc_large_al/hybrid'

In [24]:
samples_id_list = []

for subdir, dirs, files in os.walk(base_path):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith(".txt"):
            samples_id_list.append(int(file.replace("hybrid_samples","").replace(".txt","")))
            
samples_id_list.sort()




In [45]:
position_counter = {}


for sample_id in samples_id_list:
    if sample_id > 100:
        break
    with open(os.path.join(base_path, f"hybrid_samples{str(sample_id)}.txt"),"r") as file:
        samples = file.read().split("\n")
    
    for sample in samples:
        pos = "_".join(sample.split("_")[:-1])
        image_id = sample.split("_")[-1]
        
        
        if pos in position_counter.keys():
            position_counter[pos].append(image_id)
        else:
            position_counter[pos] = [image_id]
    
    print(sample_id)
    print(json.dumps(position_counter, indent=2))
    print("")


1
{
  "acdc_data2_kurt_306-3_0_20201028_pos_4": [
    "64"
  ],
  "acdc_data2_kurt_306-3_0_20201028_pos_3": [
    "184"
  ],
  "TimeLapse_2D_MIA_KC_htb1_mCitrine_flu_control_labeled_pos_3": [
    "54"
  ],
  "TimeLapse_2D_MIA_KC_htb1_mCitrine_labeled_pos_2": [
    "104",
    "104"
  ],
  "acdc_data3_Yagya_SCGE_WT_pos_6": [
    "84"
  ],
  "TimeLapse_2D_MIA_KC_htb1_mCitrine_labeled_pos_8": [
    "104"
  ],
  "acdc_data2_F016_pos_4": [
    "24"
  ],
  "acdc_data3_20210722_pos_17": [
    "94"
  ],
  "acdc_data3_Yagya_SCGE_WT_pos_2": [
    "4"
  ],
  "acdc_data2_kurt_306-3_20_20201105_pos_5": [
    "104"
  ],
  "acdc_data3_20210722_pos_13": [
    "54"
  ],
  "acdc_data3_20211029_pos_15": [
    "124"
  ],
  "acdc_data3_20211029_pos_16": [
    "84"
  ],
  "acdc_data3_20211029_pos_24": [
    "114"
  ]
}

2
{
  "acdc_data2_kurt_306-3_0_20201028_pos_4": [
    "64"
  ],
  "acdc_data2_kurt_306-3_0_20201028_pos_3": [
    "184"
  ],
  "TimeLapse_2D_MIA_KC_htb1_mCitrine_flu_control_labeled_pos_3": [